In [1]:
import numpy as np
from scipy.stats.distributions import chi2, t
from scipy.stats import chisquare, ttest_ind

### T-test для обычных метрик (средних)

In [2]:
SIZE = 5000

A_SAMP = np.random.poisson(lam=1, size=SIZE)
B_SAMP = np.random.poisson(lam=1.05, size=SIZE)

In [3]:
def covar(samp1, samp2):
    n = samp1.size
    assert n == samp2.size
    sum_square = samp1.sum() * samp2.sum()
    square_sum = (samp1 * samp2).sum()
    return (square_sum - sum_square / n) / (n - 1)


def var(samp):
    return covar(samp, samp)

In [4]:
a_mean = A_SAMP.mean()
b_mean = B_SAMP.mean()

a_mean_var = var(A_SAMP) / SIZE
b_mean_var = var(B_SAMP) / SIZE
pooled_mean_var = a_mean_var + b_mean_var

# Считаем статистику и p-value
t_stat = (a_mean - b_mean) / np.sqrt(pooled_mean_var)
df = pooled_mean_var ** 2 * (SIZE - 1) / (a_mean_var ** 2 + b_mean_var ** 2)
t_pval = t.sf(np.abs(t_stat), df=df) * 2  # Умножаем на 2, потому гипотеза обычно двусторонняя

In [5]:
# То же самое, только с помощью готовой функции
test_result = ttest_ind(A_SAMP, B_SAMP, equal_var=False)

In [6]:
# Проверяем, что действительно то же самое
print(t_stat, t_pval)
print(test_result.statistic, test_result.pvalue)

-3.0710005671041465 0.0021391572454784635
-3.0710005671041465 0.0021391572454784675


### T-test для Ratio-метрик

Пример ratio-метрики: средний чек = сумма денег во всех чеках / количество чеков.

Почему это ratio? Потому что один пользователь может сделать много чеков.

Почему нельзя просто сделать выборку по чекам? Потому что выборка должна состоять из наблюдений по пользователям — одна строка на пользователя. Иначе в выборке будет много зависимостей и оценка дисперсии будет занижена ⇒ много ложных прокрасов.

In [7]:
def generate_ratio_sample(mean, size):
    lam = np.random.randint(1, 100)
    num = np.random.poisson(lam=lam, size=size)
    den = np.random.poisson(lam=lam/mean, size=size)
    return num, den

In [8]:
SIZE = 5000
A_NUM, A_DEN = generate_ratio_sample(1, SIZE)
B_NUM, B_DEN = generate_ratio_sample(1.02, SIZE)

In [9]:
def var_ratio(num, den):
    num_mean = num.mean()
    den_mean = den.mean()
    num_var = var(num)
    den_var = var(den)
    num_den_covar = covar(num, den)
    return (
          1                 / den_mean ** 2 * num_var
        - 2 * num_mean      / den_mean ** 3 * num_den_covar
        +     num_mean ** 2 / den_mean ** 4 * den_var
    )

In [10]:
a_mean = A_NUM.sum() / A_DEN.sum()
b_mean = B_NUM.sum() / B_DEN.sum()

a_mean_var = var_ratio(A_NUM, A_DEN) / SIZE
b_mean_var = var_ratio(B_NUM, B_DEN) / SIZE
pooled_mean_var = a_mean_var + b_mean_var

# Считаем статистику и p-value
t_stat = (a_mean - b_mean) / np.sqrt(pooled_mean_var)
df = pooled_mean_var ** 2 * (SIZE - 1) / (a_mean_var ** 2 + b_mean_var ** 2)
t_pval = t.sf(np.abs(t_stat), df=df) * 2  # Умножаем на 2, потому гипотеза обычно двусторонняя

print(t_stat, t_pval)

# В scipy нет готовой функции для такого расчета. Валидировать надо с помощью bootstrap

-5.574075900230694 2.555139678738598e-08
